## Задание 1
Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
  - оценка 2 и меньше - низкий рейтинг
  - оценка 4 и меньше - средний рейтинг
  - оценка 4.5 и 5 - высокий рейтинг

In [13]:
import pandas as pd
import numpy as np
rats = pd.read_csv('ratings1.csv')
rats.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [14]:
def rat(movie):
    if movie.rat <= 2:
        return "Low"
    elif movie.rat > 2 and movie.rat < 4.5:
        return "Middle"
    else:
        return "High"
    
rats['rate']=rats.apply(rating,axis=1)
rats.head()

,userId,movieId,rating,timestamp,rate
0,1,31,2.5,1260759144,Middle
1,1,1029,3.0,1260759179,Middle
2,1,1061,3.0,1260759182,Middle
3,1,1129,2.0,1260759185,Low
4,1,1172,4.0,1260759205,Middle


## Задание 2
Используем файл keywords.csv.
Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце 'region' пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим 'undefined'.
Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

In [15]:
geo_data = {
    'Центр': ['москва', 'тула', 'ярославль'],
    'Северо-Запад': ['петербург', 'псков', 'мурманск'],
    'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

Результат классификации запишите в отдельный столбец region.

In [16]:
data = pd.read_csv('keywords.csv')
search=['|'.join(x) for x in geo_data.values()]
for i in range(len(geo_data)):
    data.loc[(data.keyword.str.contains(search[i])),'region'] = ''.join(list(geo_data.keys())[i])
data.loc[pd.isnull(data.region), 'region']='undefined'
data.head()

,keyword,shows,region
0,вк,64292779,undefined
1,одноклассники,63810309,undefined
2,порно,41747114,undefined
3,ютуб,39995567,undefined
4,вконтакте,21014195,undefined


In [18]:
#проверка 
data[data.keyword.str.contains('тула|сахалин')].head(10)

,keyword,shows,region
1579,авито тула,126168,Центр
11785,тула,23898,Центр
11795,гисметео тула,23970,Центр
14823,моя реклама тула,20375,Центр
17574,арсенал тула,17331,Центр
19783,доктор 71 электронная запись тула,15749,Центр
20821,слобода тула,15037,Центр
21445,сахалин,14494,Дальний Восток
22765,погода тула,17374,Центр
23272,киносити тула,13777,Центр


## Задание 3
Для датафрейма log из материалов занятия создайте столбец source_type по следующим правилам:
- если источник traffic_source равен yandex или google, то в source_type ставится organic
- для источников paid и email из России - ставим ad
- для источников paid и email не из России - ставим other
- все остальные варианты берем из traffic_source без изменений

In [19]:
log = pd.read_csv('visit_log.csv', sep=';')
log.head()

,timestamp,visit_id,url,region,user_id,traffic_source
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex


In [20]:
def source(inpt):
    if ('yandex'or'google')  in inpt.traffic_source :
        return 'organic'
    if (('paid'or'email')  in inpt.traffic_source) and ('Russia' in inpt.region):
        return 'ad'
    if (('paid'or'email')  in inpt.traffic_source) and ('Russia' not in inpt.region):
        return 'other'
    else:
        return inpt.traffic_source
log['source_type'] = log.apply(source,axis=1)
log.head()

,timestamp,visit_id,url,region,user_id,traffic_source,source_type
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,organic
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,organic
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,organic
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,organic


# Задание 4
В файле URLs.txt содержатся url страниц новостного сайта. Вам необходимо отфильтровать его по адресам страниц с текстами новостей. Известно, что шаблон страницы новостей имеет внутри url следующую конструкцию: /, затем 8 цифр, затем дефис. Выполните следующие действия:
1. Прочитайте содержимое файла с датафрейм
2. Отфильтруйте страницы с текстом новостей, используя метод str.contains и регулярное выражение в соответствии с заданным шаблоном

In [21]:
sort = pd.read_csv('URLs.txt', sep = ";")
sort[sort.url.str.contains('\d{8}')]

,url
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...
5,/science/36157853-nasa-sobiraet-ekstrennuyu-pr...
6,/video/36001498-poyavilis-pervye-podrobnosti-g...
7,/world/36007585-tramp-pridumal-kak-reshit-ukra...
...,...
88,/cis/35984145-kreml-prokommentiroval-soobschen...
89,/video/36071019-olimpiyskie-obekty-rio-prevrat...
90,/science/36151301-nazvano-posledstvie-zloupotr...
91,/incidents/36027330-vospitatelnitsu-zatravili-...


## Задание 5
1. Отфильтруйте датафрейм stats по поисковым запросам, которые содержат строку "погода в" и упоминают один из городов: Москва, Новосибирск, Краснодар.

In [22]:
stats = pd.read_csv('keywords.csv')
stats[(stats.keyword.str.contains('погода в'))&(stats.keyword.str.contains('москва|новосибирск|краснодар'))].head()

,keyword,shows
651,погода в краснодаре,261285
1498,погода в новосибирске,133362
2316,погода в новосибирске на 14 дней,93278
4930,погода в краснодаре на 10 дней,49354
20847,погода в новосибирске на 3 дня,15475


2. В датафрейме data создайте столбец lemmas, в котором вычислите леммы поисковых запросов из столбца keyword. Леммы должны иметь строковый тип.

In [23]:
data = pd.DataFrame({
    'keyword': ['курс гривны к рублю', 'доллары в рубли', '100 долларов в рублях', 'курс рубля'],
    'shows': [125076, 114173, 97534, 53546],
})
data

,keyword,shows
0,курс гривны к рублю,125076
1,доллары в рубли,114173
2,100 долларов в рублях,97534
3,курс рубля,53546


In [24]:
import pymystem3

ModuleNotFoundError: No module named 'pymystem3'